# Settings

In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist, squareform
from scipy.stats import spearmanr, pearsonr, ttest_ind

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from statannot import add_stat_annotation
from matplotlib import rcParams, cm, colors
from matplotlib.ticker import ScalarFormatter
from mpl_toolkits.axes_grid.inset_locator import inset_axes

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # allows to see all outputs of a cell rather then just from the last line

In [ ]:
rcParams['font.size'] = 12
rcParams['figure.figsize'] = (rcParams['figure.figsize'][0] * 1.5, rcParams['figure.figsize'][1] * 1.5)  # figure size in inches
rcParams['savefig.dpi'] = 200  # figure dots per inch
rcParams['savefig.format'] = 'png'  # {png, ps, pdf, svg}
rcParams['savefig.bbox'] = 'tight'
rcParams['savefig.transparent'] = False  # transparent background

# Data

## Genome assemblies

In [ ]:
# download "Supplementary Data 1" from Leviatan, S., Shoer, S., Rothschild, D. et al. 
# An expanded reference map of the human gut microbiome reveals hundreds of previously unknown species. 
# Nat Communications 13, 3863 (2022). https://doi.org/10.1038/s41467-022-31502-1
!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-022-31502-1/MediaObjects/41467_2022_31502_MOESM4_ESM.xlsx --no-check-certificate

In [ ]:
metadata = pd.read_excel('41467_2022_31502_MOESM4_ESM.xlsx', index_col=0)
metadata['Quality score'] = 1*metadata['Completeness'] -5*metadata['Contamination'] + 15*np.log10(metadata['N50'])

In [ ]:
# located in this section to be before filtering to species with 20 genomes
metadata[['Completeness', 'Contamination']].median()
metadata['Completeness'].min()
metadata['Contamination'].max()
metadata['type'].value_counts()/metadata.shape[0]*100

In [ ]:
all_species = metadata.loc[metadata['#assemblies'] >= 20, 'species'].tolist()
metadata = metadata[metadata['species'].isin(all_species)]

In [ ]:
phyla = metadata.dropna().set_index('species')['p'].str.split('_').str[2]
phyla_cmap = {p: cm.get_cmap('nipy_spectral')(i/(phyla.unique().shape[0])) for i, p in enumerate(sorted(phyla.unique()))}

In [ ]:
# download "Supplementary Data 3" from Leviatan, S., Shoer, S., Rothschild, D. et al. 
# An expanded reference map of the human gut microbiome reveals hundreds of previously unknown species. 
# Nat Communications 13, 3863 (2022). https://doi.org/10.1038/s41467-022-31502-1
!wget https://figshare.com/ndownloader/files/31219225 --no-check-certificate
!mv 31219225 31219225.gz
!gunzip 31219225.gz
!mv 31219225 31219225.csv

In [ ]:
# download "Supplementary Data 5" from Leviatan, S., Shoer, S., Rothschild, D. et al. 
# An expanded reference map of the human gut microbiome reveals hundreds of previously unknown species. 
# Nat Communications 13, 3863 (2022). https://doi.org/10.1038/s41467-022-31502-1
!wget https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-022-31502-1/MediaObjects/41467_2022_31502_MOESM7_ESM.xlsx --no-check-certificate

In [ ]:
abundance = pd.read_excel('41467_2022_31502_MOESM7_ESM.xlsx', index_col=0)
abundance.columns = abundance.columns.str.split('_').str[-1].astype(int)

## Pangenomes

In [ ]:
# simplified version of pan_annots of only the sub annotated genes (default core), no matter the copy number

def get_sub_annots(in_df, minimum=90, maximum=101):
    
    out_df = in_df[(in_df['%genomes'] >= minimum) & (in_df['%genomes'] < maximum)]  # is sub
    out_df = out_df[out_df['Gene'].str[:6] != 'group_']  # is annotated
    out_df['Gene'] = out_df['Gene'].str[:4]  # no matter the copy number
    out_df['Gene'] = out_df['Gene'].str.replace('_', '').str.replace('-', '').str.replace('/', '').str.replace('(', '').str.replace(' ', '')  # remove shit
    out_df = out_df.reset_index()[['SGB', 'Gene']].drop_duplicates().set_index('SGB')  # once per species

    return out_df

In [ ]:
genes_per_genomes = pd.read_excel('Supplementary file 1 - genes statistics.xlsx', sheet_name='genes_per_genomes_random_order', index_col=0)

In [ ]:
genes_type = pd.read_excel('Supplementary file 1 - genes statistics.xlsx', sheet_name='genes_type', index_col=0)
genes_type_cmap = {'Core genes': 'tab:red', 'Shell genes': 'gold', 'Cloud genes': 'tab:blue'}

In [ ]:
jaccard20 = pd.read_excel('Supplementary file 2 - genes conservation.xlsx', sheet_name='jaccard_within_species', index_col=0)
jaccard_full = pd.read_excel('Supplementary file 2 - genes conservation.xlsx', sheet_name='jaccard_between_species', index_col=[0, 1])

In [ ]:
shell_r = pd.read_excel('Supplementary file 2 - genes conservation.xlsx', sheet_name='spearman_r_within_species', index_col=0)
shell_p = pd.read_excel('Supplementary file 2 - genes conservation.xlsx', sheet_name='spearman_p_within_species', index_col=0)

In [ ]:
pan_annots = pd.read_csv('Supplementary file 5 - genes annotations.csv', index_col=0,
                         usecols=['SGB', 'Gene', 'length', '%genomes', 'ID', 'best_og_cat', 'best_og_desc', 'PFAMs', 'GENE', 'PRODUCT', 'RESISTANCE']) # reduce file size because very heavy

problematic = [449]
# remember that fastas, eggnog and antibiotic files do not include species 449

# label core, shell and cloud genes
pan_annots['Gene type'] = 'Cloud genes [0-10%]'
pan_annots.loc[pan_annots['%genomes'] >= 10, 'Gene type'] = 'Shell genes [10-90%]'
pan_annots.loc[pan_annots['%genomes'] >= 90, 'Gene type'] = 'Core genes [90-100%]'

# unite missing best_og_cat 
pan_annots['best_og_cat'] = pan_annots['best_og_cat'].replace('-', np.nan).replace('S', np.nan).fillna('Unknown')  # S - function unknown

# clean antibiotic ressitences
pan_annots['RESISTANCE'] = pan_annots['RESISTANCE'].str.replace('streptogramin_A', 'streptogramin').str.replace('streptogramin_B', 'streptogramin')
pan_annots['RESISTANCE'] = pan_annots['RESISTANCE'].str.replace('streptogramin;streptogramin', 'streptogramin').str.replace('streptogramin;streptogramin', 'streptogramin')

In [ ]:
core_annots = get_sub_annots(pan_annots)

In [ ]:
heaps = pd.read_excel('Supplementary file 3 - strain variability.xlsx', sheet_name='heaps_equation', index_col=0)
heaps_cmap = cm.ScalarMappable(norm=colors.Normalize(vmin=heaps['a'].min(), vmax=heaps['a'].max()), cmap='Blues')
heaps['colors'] = heaps['a'].apply(heaps_cmap.to_rgba)

heaps_genes = pd.read_excel('Supplementary file 3 - strain variability.xlsx', sheet_name='genes_association', index_col=0)

In [ ]:
anti = pd.read_excel('Supplementary file 4 - antibiotic resistances.xlsx', sheet_name='antibiotic_resistances', index_col=0)

# Results

## genes fold change

In [ ]:
genes_type[['n_genomes', 'n_studies', 'All genes', 'Genes in highest quality genome', 'Fold change', 'Genes in 20 genomes', 'Fold change in 20 genomes']].apply(lambda col: 
            f'{col.quantile(q=0.5, interpolation="higher"):,.2f} [{col.min():,.2f} - {col.max():,.2f}]')

In [ ]:
spearmanr(genes_type['n_genomes'], genes_type['n_studies'])

In [ ]:
_ = genes_type['All genes'].hist(log=True)
genes_type['All genes'].sort_values().tail()

In [ ]:
_ = genes_type['Fold change'].hist(log=True)
genes_type['Fold change'].sort_values().tail()

In [ ]:
uppery = 50
data = genes_type.copy().join(phyla)
data['Fold change'] = data['Fold change'].clip(upper=uppery)

# scatter
ax = sns.scatterplot(x='n_genomes', y='Fold change', data=data, alpha=0.5, hue='p', hue_order=phyla_cmap.keys(), palette=phyla_cmap)

_ = ax.set_xscale('log')
ax.xaxis.set_major_formatter(ScalarFormatter())

handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles, labels, title=r'$\bfPhylum$', bbox_to_anchor=(1, 1.02))

_ = ax.set_title(f'median={genes_type["Fold change"].median():.2f}', **{'fontsize': rcParams['font.size']})

_ = ax.set_xlabel('Number of strains')
_ = ax.set_ylabel('Genes fold change\n[#pangenome / #highest quality individual genome]')

r, p = spearmanr(genes_type['n_genomes'], genes_type['Fold change'])
_ = ax.text(x=2800, y=0, s=f'r={r:.2f}, p<1e{math.floor(math.log10(p))+1}', **{'fontsize': rcParams['font.size']})
r, p = spearmanr(genes_type['n_studies'], genes_type['Fold change'])
'n_studies', r, p

for i, (x, s) in enumerate(genes_type.loc[genes_type['Fold change'] > uppery, ['n_genomes', 'Fold change']].values):
    _ = ax.text(x=x, y=uppery, s=int(s))
    
# histogram
axins = inset_axes(ax, width=1.5, height=1.7, bbox_transform=ax.transAxes, bbox_to_anchor=(0.07, 0.95), loc='upper left')
ax2 = sns.histplot(data['Fold change in 20 genomes'], bins=np.arange(1, 7, 1), stat='percent', color='grey')

_ = ax2.set_title(f'median={genes_type["Fold change in 20 genomes"].median():.2f}', **{'fontsize': rcParams['font.size']})

_ = ax2.set_xlabel('Genes fold change\nin pangenome of\n20 strains')
_ = ax2.set_ylabel('')

_ = ax2.set_xticks([1.5, 2.5, 3.5, 4.5, 5.5])
_ = ax2.set_xticklabels([1, 2, 3, 4, 5])

_ = ax2.set_ylim(0, 55)
_ = ax2.set_yticks([0, 10, 20, 30, 40, 50])
_ = ax2.set_yticklabels(['0%', '10%', '20%', '30%', '40%', '50%'])

plt.savefig(os.path.join('figs', 'Figure 1'))

## core, shell, cloud

In [ ]:
genes_type[['Core genes', 'Shell genes', 'Cloud genes']].apply(lambda col: 
           f'{col.quantile(q=0.5, interpolation="higher"):,} [{col.min():,} - {col.max():,}]')

In [ ]:
genes_type['Cloud genes'].hist(log=True)
genes_type['Cloud genes'].sort_values()

In [ ]:
_ = genes_type.loc[genes_type['n_genomes'] < 30, 'Cloud genes'].hist(bins=50, color=genes_type_cmap['Cloud genes'])
_ = plt.xlabel('Number of cloud genes in species with less than 30 strains')
_ = plt.ylabel('Count')
_ = plt.savefig(os.path.join('figs', '4reviewer - L citreum'))

In [ ]:
genes_type.sort_values('Cloud genes')

In [ ]:
genes_type.sort_values('Cloud genes').iloc[0]

In [ ]:
metadata[metadata['species'] == genes_type['Cloud genes'].idxmin()].dropna().T

In [ ]:
metadata.loc[metadata['species'] == genes_type['Cloud genes'].idxmin(), 'source'].value_counts()

In [ ]:
metadata.loc[(metadata['species'] == genes_type['Cloud genes'].idxmin()) & (metadata['source'] == 'Segata')].T.dropna()

In [ ]:
abundance.shape

In [ ]:
abundance.loc[:, genes_type["Cloud genes"].idxmin()].dropna().sort_values()*100#.shape

In [ ]:
abundance.loc[:, genes_type["Cloud genes"].idxmax()].dropna().sort_values()*100#.shape

In [ ]:
data = genes_type.set_index('n_genomes', append=True)[['Core genes', 'Shell genes', 'Cloud genes']].stack().reset_index()
data.columns = ['species', 'n_genomes', 'Gene frequency', 'n_genes']

d = {'Core genes': 'Core genes [90-100%]\n' + 
     f'r={spearmanr(genes_type["Core genes"], genes_type["n_genomes"])[0]:.2f}, ' + 
     f'p={spearmanr(genes_type["Core genes"], genes_type["n_genomes"])[1]:.2f}',
     'Shell genes': 'Shell genes [10-90%]\n' + 
     f'r={spearmanr(genes_type["Shell genes"], genes_type["n_genomes"])[0]:.2f}, ' + 
     f'p={spearmanr(genes_type["Shell genes"], genes_type["n_genomes"])[1]:.2f}',
     'Cloud genes': 'Cloud genes [0-10%]\n' + 
     f'r={spearmanr(genes_type["Cloud genes"], genes_type["n_genomes"])[0]:.2f}, ' + 
     f'p<1e{math.floor(math.log10(spearmanr(genes_type["Cloud genes"], genes_type["n_genomes"])[1]))+1}'}
data['Gene frequency'] = data['Gene frequency'].replace(d)

ax = sns.scatterplot(x='n_genomes', y='n_genes', hue='Gene frequency', style='Gene frequency', data=data, 
                     alpha=0.5, markers=['^', '>', 'v'], palette={d[k]:v for k, v in genes_type_cmap.items()})

handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(reversed(handles), reversed(labels), title=r'$\bfGene\;frequency$')

_ = plt.xlabel('Number of strains')
_ = plt.ylabel('Number of genes')

_ = plt.xscale('log')
_ = plt.yscale('log')
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.yaxis.set_major_formatter(ScalarFormatter())

_ = ax.text(x=genes_type.loc[genes_type['Cloud genes'].idxmin(), 'n_genomes'], y=genes_type['Cloud genes'].min(), s=' L. citreum', ha='left', **{'fontproperties':{"style": "italic"}})
_ = ax.text(x=genes_type.loc[genes_type['Cloud genes'].idxmax(), 'n_genomes'], y=genes_type['Cloud genes'].max(), s=' P. vulgatus', ha='right', **{'fontproperties':{"style": "italic"}})

plt.savefig(os.path.join('figs', 'Figure 2'))

In [ ]:
{'Core genes': 'Core genes [90-100%]\n' + 
 f'r={spearmanr(genes_type["Core genes"], genes_type["n_studies"])[0]:.2f}, ' + 
 f'p={spearmanr(genes_type["Core genes"], genes_type["n_studies"])[1]:.2f}',
 'Shell genes': 'Shell genes [10-90%]\n' + 
 f'r={spearmanr(genes_type["Shell genes"], genes_type["n_studies"])[0]:.2f}, ' + 
 f'p={spearmanr(genes_type["Shell genes"], genes_type["n_studies"])[1]:.4f}',
 'Cloud genes': 'Cloud genes [0-10%]\n' + 
 f'r={spearmanr(genes_type["Cloud genes"], genes_type["n_studies"])[0]:.2f}, ' + 
 f'p<1e{math.floor(math.log10(spearmanr(genes_type["Cloud genes"], genes_type["n_studies"])[1]))+1}'}

In [ ]:
data = pan_annots.loc[~pan_annots.index.isin(problematic), ['length', 'Gene type']]
data['Gene type'] = data['Gene type'].str.split('[').str[0].str[:-1]
data = data.groupby(['SGB', 'Gene type'])['length'].mean().reset_index('Gene type')
data.groupby('Gene type')['length'].apply(lambda col: f'{col.mean():.2f}±{col.std():.2f}').loc[genes_type_cmap.keys()]

order = list(genes_type_cmap.keys())

ax = sns.boxplot(x='Gene type', y='length', data=data, order=order, showfliers=False, dodge=False, palette=genes_type_cmap)

_ = ax.set_xlabel('')
_ = ax.set_ylabel('Gene length')
# _ = ax.set_ylim(0, 2500)

box_pairs = []
for i, o1 in enumerate(order):
    for o2 in order[i+1:]:
        box_pairs.append((o1, o2))
_ = add_stat_annotation(ax, x='Gene type', y='length', data=data, order=order,
                        box_pairs=box_pairs, test='t-test_ind', comparisons_correction='bonferroni', text_format='simple', verbose=0)

_ = plt.savefig(os.path.join('figs', '4reviewer - genes length'))

## strain variability

In [ ]:
data = (~abundance.isna()).sum().to_frame('prevalence').join(abundance.mean().to_frame('abundance')).join(np.log10(abundance.mean()).to_frame('abundance_log'))
data = data.loc[all_species].join(heaps)

In [ ]:
_ = sns.scatterplot(x='prevalence', y='a', data=data)
spearmanr(data['prevalence'], data['a'])

In [ ]:
_ = sns.scatterplot(x='abundance', y='a', data=data.fillna(0))
spearmanr(data['abundance'].fillna(0), data['a'])

In [ ]:
_ = sns.scatterplot(x='abundance_log', y='a', data=data.fillna(0))
spearmanr(data['abundance_log'].fillna(0), data['a'])

In [ ]:
core_annots[core_annots['Gene'].isin(heaps_genes.index)].groupby('Gene').apply(len).sort_values()

In [ ]:
core_annots[core_annots['Gene'].isin(heaps_genes[heaps_genes['p_bonferroni'] < 0.05].index)].groupby('Gene').apply(len).sort_values()

In [ ]:
heaps_genes[heaps_genes['p_bonferroni'] < 0.05].sort_values('p')

In [ ]:
heaps_genes[(heaps_genes['p_bonferroni'] < 0.05) & (heaps_genes['s'] > 0)].sort_values('p').shape[0]
heaps_genes[(heaps_genes['p_bonferroni'] < 0.05) & (heaps_genes['s'] < 0)].sort_values('p').shape[0]

In [ ]:
data = genes_per_genomes.stack().reset_index()
data.columns = ['n_genomes', 'species', 'n_genes']
data['n_genomes'] = data['n_genomes'] + 1
data = data[data['n_genomes'] <= 1000]

fig = plt.figure()

# lines
ax = sns.lineplot(x='n_genomes', y='n_genes', hue='species', data=data, 
                 hue_order=heaps['a'].sort_values().index.tolist(), palette=heaps['colors'].to_dict(), legend=False)

_ = ax.figure.colorbar(heaps_cmap, cax=fig.add_axes([0.145, 0.8, 0.25, 0.05]), **{'orientation': 'horizontal', 'label': 'Strain variability [a]', 'ticks': [0.1, 0.3, 0.5, 0.7]})

_ = ax.set_xlabel('Number of strains')
_ = ax.set_ylabel('Number of genes')

_ = ax.text(x=1000, y=data.loc[data['species'] == 959, 'n_genes'].iloc[-1], s='  E. flexneri', **{'fontproperties':{"style": "italic"}})

plt.savefig(os.path.join('figs', 'Figure 3'))

In [ ]:
color_high = heaps.loc[heaps['a'] == heaps['a'].quantile(0.9, interpolation='lower'), 'colors'].iloc[0]
color_low = heaps.loc[heaps['a'] == heaps['a'].quantile(0.1, interpolation='lower'), 'colors'].iloc[0]

for gene in heaps_genes[heaps_genes['p_bonferroni'] < 0.05].sort_values('p').index:

    have_gene = core_annots[core_annots['Gene'] == gene].index.to_list()
    miss_gene = list(set(all_species) - set(have_gene))

    heaps[gene] = False
    heaps.loc[have_gene, gene] = True
    
    median_have = heaps.loc[have_gene, 'a'].sort_values().iloc[int(len(have_gene)/2)]
    median_miss = heaps.loc[miss_gene, 'a'].sort_values().iloc[int(len(miss_gene)/2)]
    
    _ = plt.figure(figsize=(2, 3.5))
    _ = sns.boxplot(x=gene, y='a', data=heaps, showfliers=False, order=[True, False], 
                    palette=[color_high, color_low] if median_have > median_miss else [color_low, color_high])
    
    _ = plt.xticks([0, 1], [f'Have\ngene\nn={len(have_gene):,}', f'Missing\ngene\nn={len(miss_gene):,}'])
    _ = plt.xlabel('')
    
    _ = plt.ylim(0, 1)
    _ = plt.ylabel('Strain variability [a]')
    
    _ = plt.title(f'{gene} gene')
    _ = plt.text(x=0.1, y=0.9, s=f'p<1e{math.floor(math.log10(heaps_genes.loc[[gene], "p_bonferroni"].iloc[0]))+1}')
    
    plt.savefig(os.path.join('figs', 'heaps genes', gene))
    plt.close()

## antibiotic resistances

In [ ]:
# sorter = (anti == 0).sum().to_frame('no resistance').join(phyla).sort_values(['p', 'no resistance'])
# data = anti.loc[(anti > 0).sum(axis=1).sort_values(ascending=False).index, sorter.index] # just sorting
data = anti.copy()

specific_names = ['polymyxin|colistin', 'aztreonam', 'tigecycline', 'fosfomycin', 'daptomycin', 'linezolid', 'vancomycin']
class_names = ['carbapenem']#, 'cephalosporin']
# 'carbapenem|meropenem' and 'cephalosporin' are excluded because they are antibiotic classes
# in cephalosporin we should only take 4th and 5th generation, however the annotations are not specific enough to distinguish
# data = data.loc[data.index[~data.index.isin(specific_names)].tolist() + data.index[data.index.isin(class_names) | data.index.isin(specific_names)].tolist()]

col_colors = phyla[anti.columns].map(phyla_cmap)

ax = sns.clustermap(data=data, mask=data==0, vmin=0, vmax=100, cmap='Reds', cbar_kws={'label': '% Strains with antibiotic resistance'},
                    col_cluster=False, row_cluster=False,
                    figsize=(20, 8), cbar_pos=(0.88, 0.025, 0.02, 0.4), dendrogram_ratio=0.0001,
                    xticklabels=False, col_colors=col_colors,
                    yticklabels=[f'{l[0].upper()}{l[1:]}' for l in 
                                 data.index.str.replace('disinfecting_agents_and_antiseptics', 'disinfecting&antiseptics').str.replace('_', ' ').str.replace('polymyxin\|colistin', 'polymyxin')])

handles = [Patch(facecolor=color) for color in phyla_cmap.values()]
_ = ax.ax_heatmap.legend(handles, phyla_cmap, title=r'$\bfPhylum$', bbox_to_anchor=(1.14, 1.05))

_ = ax.ax_col_colors.set_yticks([])

_ = ax.ax_heatmap.yaxis.label_position = 'left'
_ = ax.ax_heatmap.yaxis.tick_left()

_ = ax.ax_heatmap.axhline(y=0, color='black', linewidth=1.5)
_ = ax.ax_heatmap.axhline(y=ax.ax_heatmap.get_ylim()[0], color='black', linewidth=1.5)
_ = ax.ax_heatmap.axhline(y=ax.ax_heatmap.get_ylim()[0]-data.index.intersection(specific_names+class_names).shape[0], color='black', linewidth=1.5)
_ = ax.ax_heatmap.axvline(x=0, color='black', linewidth=1.5)
_ = ax.ax_heatmap.axvline(x=ax.ax_heatmap.get_xlim()[1], color='black', linewidth=1.5)

_ = ax.ax_heatmap.set_xlabel('Species', labelpad=-80)
_ = ax.ax_heatmap.set_ylabel('Last resort                                           Antibiotic class\nantibiotics', loc='bottom')

labels2show = {
    959: '\nEnterobacteriaceae',
    
    807: 'S. maltophilia', # Stenotrophomonas
    796: 'C. kerstersii\n\n\n', # Comamonas
    
    2746: 'B. wadsworthia', # Bilophila
    1190: 'S. epidermidis\n\n', # Staphylococcus
    
    477: 'Bacteroides',
    1113: 'Enterococcus\n\n',
    
    482: 'B. fragilis', # Bacteroides
    1374: 'C. difficile', # Clostridioides
    1880: 'V. seminalis', # Veillonella
}

_ = ax.ax_heatmap.set_xticks([data.columns.get_loc(l) for l in labels2show.keys()])
_ = ax.ax_heatmap.set_xticklabels(list(labels2show.values()), **{'ha':'left', 'fontproperties':{"style": "italic"}, 'rotation':-30})

plt.savefig(os.path.join('figs', 'Figure 4'))

In [ ]:
card_classes = ['albendazole-like',  'alkaloids',  'allylamine',  'aminocoumarin',  'aminoglycoside',  'ansamycin',  'antibacterial_free_fatty_acids',  'bicyclomycin-like',  'carbapenem',  'cephalosporin',  'cephamycin',  'cyclomarin',  'cycloserine-like',  'diaminopyrimidine',  'diarylquinoline',  'disinfecting_agents_and_antiseptics',  'echinocandin',  'elfamycin',  'fluoroquinolone',  'fusidane',  'glycopeptide',  'glycylcycline',  'imidazole',  'ionophore',  'isoniazid-like',  'lincosamide',  'macrocyclic',  'macrolide',  'metallophores',  'monobactam',  'mupirocin-like',  'nitrofuran',  'nitroimidazole',  'nucleoside',  'nybomycin-like',  'organoarsenic',  'orthosomycin',  'oxacephem',  'oxazolidinone',  'pactamycin-like',  'penam',  'penem',  'peptide',  'phenicol',  'phosphonic_acid',  'pleuromutilin',  'polyamine',  'polyene',  'polyketide',  'protegrin-1-like',  'pyrazine',  'quinone',  'rifamycin',  'salicylic_acid',  'streptogramin',  'sulfonamide',  'sulfone',  'terpene',  'tetracenomycin',  'tetracycline',  'thiazolide',  'thioamide',  'thiosemicarbazone',  'triazaacenaphthylene',  'triazole',  'zoliflodacin-like']

In [ ]:
len(card_classes)

In [ ]:
len(set(card_classes) - set(anti.index))

In [ ]:
print(set(card_classes) - set(anti.index))

In [ ]:
data = pan_annots[['%genomes', 'GENE', 'PRODUCT', 'RESISTANCE', 'Gene type']].dropna()

In [ ]:
data.shape

In [ ]:
pan_annots['GENE'].dropna().str[:4].str.replace('_', '').str.replace('-', '').str.replace('/', '').str.replace('(', '').str.replace(' ', '').value_counts()

In [ ]:
data['Gene type'].value_counts()/data.shape[0]*100

In [ ]:
data.index.unique().shape

In [ ]:
flat = anti.loc[anti.index.difference(specific_names)].drop_duplicates().replace(0, np.nan).stack().sort_values()

In [ ]:
pd.cut(flat, bins=[0, 10, 90, 100]).value_counts()

In [ ]:
flat[flat > 90].reset_index().join(phyla, on='level_1')['p'].value_counts()

In [ ]:
flat[flat > 90].index.get_level_values(1).value_counts().to_frame('vc').join(phyla).join(metadata.dropna().set_index('species')[['f', 'g', 's']])

In [ ]:
(anti.loc[anti.index.difference(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] == 0).sum(axis=1).to_frame('=0').join(
(anti.loc[anti.index.difference(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] <= 90).sum(axis=1).to_frame('<=90')).join(
(anti.loc[anti.index.difference(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] > 90).sum(axis=1).to_frame('>90')).sort_values(['<=90', '=0', '>90'], ascending=False)

In [ ]:
(anti.loc[anti.index.intersection(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] == 0).sum(axis=1).to_frame('=0').join(
(anti.loc[anti.index.intersection(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] <= 90).sum(axis=1).to_frame('<=90')).join(
(anti.loc[anti.index.intersection(specific_names).drop_duplicates(), sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())] > 90).sum(axis=1).to_frame('>90')).sort_values(['<=90', '=0', '>90'], ascending=False)

In [ ]:
anti.loc['sulfonamide', sorted(metadata.loc[metadata['f'] == 'f__Enterobacteriaceae', 'species'].tolist())]

In [ ]:
# cephalosporin                          19, non Proteobacteria 10, most Bacteroides, 477-498
# macrolide                              17, non Proteobacteria 6, most Enterococcus, 1108-1113
# fluoroquinolone                        17, non Proteobacteria 5, most Enterococcus, 1108-1113

In [ ]:
flat[flat > 90].reset_index().join(
    phyla, on='level_1', how='inner').join(#.replace('Proteobacteria', np.nan).dropna()
    metadata.dropna().set_index('species')[['f', 'g', 's']], on='level_1', how='inner')['Antibiotic/Species ID'].value_counts()
# .set_index('RESISTANCE').loc[['fluoroquinolone']].sort_values(['p', 'f', 'g', 's'])

In [ ]:
last = anti.loc[anti.index.intersection(class_names+specific_names)].drop_duplicates().replace(0, np.nan).stack().sort_values()
last

In [ ]:
last[last > 90].sort_index(level=1)

In [ ]:
metadata.set_index('species')[['p', 'c', 'o', 'f', 'g', 's']].dropna().loc[last[last > 90].index.get_level_values(1).unique()].sort_index()

In [ ]:
last[(last > 50) & (last <= 90)].sort_index()

In [ ]:
metadata.set_index('species')[['p', 'c', 'o', 'f', 'g', 's']].dropna().loc[last[(last > 50) & (last <= 90)].index.get_level_values(1).unique()].sort_index()

In [ ]:
genes_type.loc[flat[flat > 90].index.get_level_values(1).unique(), ['n_genomes', 'n_studies']].sort_values('n_studies')

In [ ]:
# Segal is not hospitalized but can have disease

# Alm "healthy FMT donors recruited in the Boston area"
# Xiao "fecal samples of healthy individuals"
# Lawley "Fecal samples were collected from 20 healthy adults"

# Kyrpides "phenotypically and demographically diverse human subjects"
# Segata "spanning 46 datasets from multiple populations, body sites, and host ages"

# Supplementary

In [ ]:
pvalue_thresholds = [[10.0**-i, f'{10.0**-i:.0e}'] for i in np.arange(500, 3, -1)] + [[1e-3, "0.001"], [1e-2, "0.01"]]

## quality

In [ ]:
(metadata.groupby('species')['type'].apply(lambda s: (s != 'Short-read MAG').sum()) >= 20).sum()

In [ ]:
params = metadata.groupby('species')[['Completeness', 'Contamination']].median().sort_index()
params.columns = ['% Completeness', '% Contamination']

In [ ]:
results = genes_type[['Fold change', 'Core genes', 'Shell genes', 'Cloud genes']].join(heaps['a']).sort_index()
results.columns = ['Genes fold change', '# Core genes', '# Shell genes', '# Cloud genes', 'Strain variability [a]']

In [ ]:
uppery = 50
data = results.join(params)
data['Genes fold change'] = data['Genes fold change'].clip(upper=uppery)

grid = sns.pairplot(data=data, x_vars=params.columns, y_vars=results.columns,
                    kind='scatter', plot_kws={'color': 'grey', 'alpha': 0.5},
                    hue=None, hue_order=None, palette=None)

# .set_facecolor('red') # used to understand in which axis i am

# x-axis
_ = grid.figure.axes[0].set_xlim(70, 100)

_ = grid.figure.axes[1].set_xlim(0, 5)
_ = grid.figure.axes[1].set_xticks([0, 1, 2, 3, 4, 5])

# y-axis
_ = grid.figure.axes[0].collections[0].set_color(phyla.sort_index().map(phyla_cmap).tolist())
_ = grid.figure.axes[1].collections[0].set_color(phyla.sort_index().map(phyla_cmap).tolist())

_ = grid.figure.axes[2].set_yscale('log')
_ = grid.figure.axes[4].set_yscale('log')
_ = grid.figure.axes[6].set_yscale('log')

_ = grid.figure.axes[8].set_ylim(0, 1)
_ = grid.figure.axes[8].collections[0].set_color(heaps.sort_index()['colors'].tolist())
_ = grid.figure.axes[9].collections[0].set_color(heaps.sort_index()['colors'].tolist())

def add_text(x, y, ax=None, **kws):
    r, p = pearsonr(x, y)
    if p < 0.05:
        ax = plt.gca()
        ax.annotate(f'EV={r*r*100:.2f}%', xy=(0.1, 0.9), xycoords=ax.transAxes)
        
_ = grid.map(add_text)

_ = grid.figure.axes[1].legend(handles=[Line2D([0], [0], linewidth=0, label=l, color=c, marker='o') for l, c in phyla_cmap.items()], title=r'$\bfPhylum$', bbox_to_anchor=(1, 1.02))
_ = grid.figure.colorbar(heaps_cmap, cax=grid.figure.add_axes([0.99, 0.076, 0.07, 0.11]), **{'orientation': 'vertical', 'label': 'Strain variability [a]', 'ticks': [0.1, 0.3, 0.5, 0.7]})

plt.savefig(os.path.join('figs', 'Figure S1'))

## genes conservation

In [ ]:
m = metadata.dropna().set_index('species')[['genus', 'family']].join(phyla.to_frame('phylum'))

jaccard_full = jaccard_full.join(m, on='Species ID 1').join(m, on='Species ID 2', lsuffix=1, rsuffix=2)
jaccard_full['genus'] = jaccard_full['genus1'] == jaccard_full['genus2']
jaccard_full['family'] = jaccard_full['family1'] == jaccard_full['family2']
jaccard_full['phylum'] = jaccard_full['phylum1'] == jaccard_full['phylum2']
jaccard_full.loc[jaccard_full['phylum'], 'phylum'] = jaccard_full.loc[jaccard_full['phylum'], 'phylum1']

In [ ]:
_, (ax1, ax2, ax3, ax4) = plt.subplots(ncols=4, sharey=True, gridspec_kw={'width_ratios': [1, 2, 2, 4]})

# phylum
level = 'Phylum'
order = (phyla.value_counts() >= 20).replace(False, np.nan).dropna().index
order = jaccard_full[jaccard_full['phylum'].isin(order)].groupby('phylum').apply(lambda p: p['Jaccard index'].median()).sort_values(ascending=False).index
data = jaccard_full[jaccard_full['phylum'].isin(order)]
ax4 = sns.boxplot(x='phylum', y='Jaccard index', hue='phylum', data=data, 
                  ax=ax4, showfliers=False, dodge=False, 
                  order=order, hue_order=phyla_cmap.keys(), palette=phyla_cmap)
_ = ax4.legend([],[], frameon=False)
_ = ax4.set_xticklabels(order, rotation=-45, ha='left')
_ = ax4.set_xlabel('')
_ = ax4.set_xticklabels([f'{label.get_text()}\nn={sum(data["phylum"] == label.get_text()):,}' for label in ax4.get_xticklabels()])
_ = ax4.set_ylabel('')
_ = ax4.set_title('Phylum')
box_pairs = []
for i, o1 in enumerate(order):
    col = data.loc[data[level.lower()] == o1, 'Jaccard index']
    f'{o1} {col.mean():.2f}±{col.std():.2f}'

    for o2 in order[i+1:]:
        box_pairs.append((o1, o2))
_ = add_stat_annotation(ax4, x='phylum', y='Jaccard index', data=data, 
                        order=order, box_pairs=box_pairs, test='t-test_ind', comparisons_correction='bonferroni', text_format='simple', verbose=0, pvalue_thresholds=pvalue_thresholds)

# genus and family
for ax, level in zip([ax3, ax2], ['Family', 'Genus']):
    order = [f'Same\n{level.lower()}', f'Different\n{level.lower()}']
    data = jaccard_full.replace(True, f'Same\n{level.lower()}').replace(False, f'Different\n{level.lower()}')
    ax = sns.boxplot(x=level.lower(), y='Jaccard index', hue=level.lower(), data=data, 
                      ax=ax, showfliers=False, dodge=False,
                      order=order, hue_order=order, palette='Reds_r')
    _ = ax.legend([],[], frameon=False)
    _ = ax.set_xlabel('')
    _ = ax.set_xticklabels([f'{label.get_text()}\n{"n=" if i == 0 else "   "}{sum(data[level.lower()] == label.get_text()):,}' for i, label in enumerate(ax.get_xticklabels())])
    _ = ax.set_ylabel('')
    _ = ax.set_title(level)
    _ = add_stat_annotation(ax, x=level.lower(), y='Jaccard index', data=data, 
                            order=order, box_pairs=[order], test='t-test_ind', comparisons_correction=None, text_format='simple', verbose=0, pvalue_thresholds=pvalue_thresholds)
    
    col = data.loc[data[level.lower()] == f'Same\n{level.lower()}', 'Jaccard index']
    f'{level} {col.mean():.2f}±{col.std():.2f}'


# species
level = 'Species'
ax1 = sns.boxplot(y=jaccard20.mean(axis=1), 
                  ax=ax1, showfliers=False, dodge=False,
                  palette=['tab:Red'])
_ = ax1.legend([],[], frameon=False)
_ = ax1.set_xticklabels([f'Pangenome of\n20 strains\nn={jaccard20.shape[0]:,}'])
_ = ax1.set_xlabel('')
_ = ax1.set_ylabel('Core genes Jaccard index')
_ = ax1.set_title('Species')

col = jaccard20.mean(axis=1)
f'{level} {col.mean():.2f}±{col.std():.2f}'

_ = plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

plt.savefig(os.path.join('figs', 'Figure S2'))

In [ ]:
for col in jaccard20.columns:
    s, p = ttest_ind(jaccard20[col], jaccard_full.loc[jaccard_full['genus'], 'Jaccard index'])
    col, s, p

In [ ]:
i = shell_r.mean().sort_values().index[5]
f'{shell_r.mean()[i]:.2f}±{shell_r.std()[i]:.2f} {shell_p.loc[:, i].quantile(q=0.5, interpolation="higher"):.2e}'

In [ ]:
shell_r[shell_p > 0.05/len(all_species)] = np.nan

In [ ]:
i = shell_r.mean().sort_values().index[5]
f'{shell_r.mean()[i]:.2f}±{shell_r.std()[i]:.2f} {shell_p.loc[:, i].quantile(q=0.5, interpolation="higher"):.2e}'

## functional categories

In [ ]:
# remember that fastas, eggnog and antibiotic files do not include species 449, 477 is a cloud gene outlier
cats = pan_annots.loc[~pan_annots.index.isin(problematic), 'best_og_cat'].value_counts()
# divides annotations with multiple categories to their individual category
for cat in cats.index:
    if (len(cat) > 1) & (cat != 'Unknown'):
        for c in cat:
            cats.loc[c] = cats.loc[c] + cats.loc[cat]/len(cat)
cats = cats.loc[(cats.index.str.len() == 1) | (cats.index == 'Unknown')]
# taking only categories that are over 1%
cats = cats/cats.sum()*100
cats = cats[cats >= 1].index.tolist()
len(cats)

index = []
for s in all_species:
    for c in cats:
        index.append((s, c))

data = pd.DataFrame(index=pd.MultiIndex.from_tuples(index, names=('SGB', 'best_og_cat')), columns=['Core genes [90-100%]', 'Shell genes [10-90%]', 'Cloud genes [0-10%]'])
for gene_type in data.columns:
    vc = pan_annots.loc[~pan_annots.index.isin(problematic) & (pan_annots['Gene type'] == gene_type), 'best_og_cat'].reset_index().value_counts()
    # divides annotations with multiple categories to their individual category
    for s in all_species:
        if s in problematic:
            continue
        for cat in vc.loc[s].index:
            if (len(cat) > 1) & (cat != 'Unknown'):
                for c in cat:
                    vc.loc[(s, c)] = (vc.loc[(s, c)] if c in vc.loc[s].index else 0) + vc.loc[(s, cat)]/len(cat)
    # filter to the 1% categories
    vc = vc[vc.index.get_level_values('best_og_cat').isin(cats)]
    vc = vc.unstack('SGB')
    vc = pd.concat([vc[vc.index != 'Unknown']/vc[vc.index != 'Unknown'].sum()*100,  # categories are divided by their sum without unknowns
                    vc.loc[['Unknown']]/vc.sum()*100])  # unknowns are divided by everything, known and unknown
    vc = vc.T.stack()
    data.loc[vc.index, gene_type] = vc.values
    
data = data.dropna(how='all').fillna(0)
data = data.stack().reset_index()
data.columns = ['SGB', 'best_og_cat', 'Gene frequency', 'Percentage']
data['Gene frequency'] = data['Gene frequency'].str.split('[').str[0].str[:-1]

In [ ]:
_ , (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 8), gridspec_kw={'width_ratios': [1, len(cats)-1], 'wspace': 0.1})

hue_order = list(genes_type_cmap.keys())
hatches = ['///', '//', '/']

# Unknown
_ = sns.barplot(x='best_og_cat', y='Percentage', hue='Gene frequency', data=data[data['best_og_cat'] == 'Unknown'], hue_order=hue_order, ax=ax1, alpha=0.5, palette=genes_type_cmap, errorbar='sd')
_ = ax1.set_xlabel('')
_ = ax1.set_ylim(0, 60)

for i, bar in enumerate(ax1.patches):
    bar.set_hatch(hatches[i % len(hue_order)])

box_pairs = [(('Unknown', hue_order[g%3]), ('Unknown', hue_order[(g+1)%3])) for g in [1, 2, 3]]
_ = add_stat_annotation(ax1, x='best_og_cat', y='Percentage', hue='Gene frequency', data=data[data['best_og_cat'] == 'Unknown'], hue_order=hue_order,
                        box_pairs=box_pairs, test='t-test_paired', comparisons_correction='bonferroni', text_format='simple', loc='outside', verbose=0, pvalue_thresholds=pvalue_thresholds)

_ = ax1.legend([],[], frameon=False)

for g in hue_order:
    col = data.loc[(data['best_og_cat'] == 'Unknown') & (data['Gene frequency'] == g), 'Percentage']
    f'{g} {col.mean():.2f}±{col.std():.2f}'

# Category
order = data[data['Gene frequency'] == 'Core genes'].groupby('best_og_cat')['Percentage'].mean().sort_values(ascending=False).drop('Unknown').index

_ = sns.barplot(x='best_og_cat', y='Percentage', hue='Gene frequency', data=data, order=order, hue_order=hue_order, ax=ax2, alpha=0.5, palette=genes_type_cmap, errorbar='sd')
_ = ax2.set_xlabel('Functional category')
_ = ax2.set_ylim(0, 30)

for i, bar in enumerate(ax2.patches):
    bar.set_hatch(hatches[int(i/len(order))])

box_pairs = [((c, hue_order[g%3]), (c, hue_order[(g+1)%3])) for g in [1, 2, 3] for c in cats[1:]]
_ = add_stat_annotation(ax2, x='best_og_cat', y='Percentage', hue='Gene frequency', data=data, order=order,
                        box_pairs=box_pairs, test='t-test_paired', comparisons_correction='bonferroni', text_format='simple', loc='outside', verbose=0, pvalue_thresholds=pvalue_thresholds)

handles, labels = ax2.get_legend_handles_labels()
_ = ax2.legend(handles, labels, title=r'$\bfGene\;frequency$', fancybox=True)

plt.savefig(os.path.join('figs', 'Figure S3'))

In [ ]:
data[data['Gene frequency'] == 'Core genes'].groupby('best_og_cat')['Percentage'].mean().sort_values(ascending=False)

In [ ]:
data[data['Gene frequency'] == 'Shell genes'].groupby('best_og_cat')['Percentage'].mean().sort_values(ascending=False)

In [ ]:
data[data['Gene frequency'] == 'Cloud genes'].groupby('best_og_cat')['Percentage'].mean().sort_values(ascending=False)

In [ ]:
# # for reviewer
# pan_annots = pd.read_csv('Supplementary file 5 - genes annotations.csv', index_col=0)
# pan_annots = pan_annots.loc[~pan_annots.index.isin(problematic)]
# pan_annots = pan_annots.replace('-', np.nan).replace('S', np.nan).replace('ko00000', np.nan)  # S - function unknown

# cols = ['eggNOG OGs', 'narr_og_name', 'narr_og_cat', 'narr_og_desc', 'best_og_name', 'best_og_cat', 'best_og_desc', 
#         'KEGG_ko', 'KEGG_Pathway', 'KEGG_Module', 'KEGG_Reaction', 'KEGG_rclass', 'KEGG_TC', 
#         'PFAMs',
#         'BRITE', 'EC', 'CAZy', 'GOs', 'BiGG_Reaction']
                                                                                          
# for col in cols:
#     vc = pan_annots[col].value_counts().index
#     print(f'{col}\t{(~pan_annots[col].isna()).mean()*100:.2f}\t{pan_annots[col].unique().shape[0]:,}\t{vc[0]}')

## strain variability

In [ ]:
data = heaps.join(phyla)
order = data.groupby('p')['a'].median().sort_values(ascending=False).index

ax = sns.boxplot(x='p', y='a', hue='p', data=data, 
                 order=order, hue_order=phyla_cmap.keys(), palette=phyla_cmap, showfliers=False, dodge=False)

handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles, labels, title=r'$\bfPhylum$', bbox_to_anchor=(1, 1.02))

box_pairs = []
big_phyla = (phyla.value_counts() >= 20).replace(False, np.nan).dropna().index
for i, p1 in enumerate(big_phyla):
    col = data.loc[data['p'] == p1, 'a']
    f'{p1} {col.mean():.2f}±{col.std():.2f}'

    for p2 in big_phyla[i+1:]:
        box_pairs.append((p1, p2))
_ = add_stat_annotation(ax, x='p', y='a', data=data, 
                        order=order, box_pairs=box_pairs, test='t-test_ind', comparisons_correction='bonferroni', text_format='simple', verbose=0, line_offset=0.01, pvalue_thresholds=pvalue_thresholds)

_ = ax.set_yticks(np.arange(0, 1.2, 0.2))
_ = ax.set_xticklabels([f'{label.get_text()}\nn={sum(phyla == label.get_text()):,}' for label in ax.get_xticklabels()], rotation=-45, ha='left')

_ = ax.set_xlabel('')
_ = ax.set_ylabel('Strain variability [a]')

plt.savefig(os.path.join('figs', 'Figure S4'))

# Methods

In [ ]:
# this is the code used to implement the methods, it can not run by useres

In [ ]:
# function to read the roary gene_presence_absence.csv files
def read_gene_presence_absence(species):
    file = os.path.join('run', f'Rep_{species}', 'roary_3.13.0', 'gene_presence_absence.csv')

    if species == 449:  # this species is very heavy and thus loaded in parts
        all_chunks = []
        for chunk in pd.read_csv(file, dtype=str, index_col=0, chunksize=5000):#10000
            all_chunks.append((~chunk.iloc[:, 13:].isna()))
        del chunk
        genes = pd.concat(all_chunks)
        del all_chunks
    else:
        genes = (~pd.read_csv(file, dtype=str, index_col=0).iloc[:, 13:].isna())
        
    return genes

## quality

In [ ]:
# # quality = pd.read_csv(os.path.join('run', 'quality2', 'quality_report.tsv'), sep='\t')
# quality = pd.read_csv(os.path.join('run', 'quality2', 'quality_report_excluded_files.tsv'), sep='\t')
# # the file have duplicate headers
# quality = quality[quality['Name'] != 'Name'].set_index('Name')
# quality['Completeness'] = quality['Completeness_General']
# quality.loc[quality['Completeness_Model_Used'] == 'Neural Network (Specific Model)', 'Completeness'] = quality['Completeness_Specific']
# quality = quality.rename(columns={'Contig_N50': 'N50 (contigs)'})

In [ ]:
# quality.shape

In [ ]:
# f"70% {((quality['Completeness'].astype(float) >= 70) & (quality['Contamination'].astype(float) <= 5)).sum()}"#".mean()*100:.2f}"
# f"50% {((quality['Completeness'].astype(float) >= 50) & (quality['Contamination'].astype(float) <= 10)).sum()}"#.mean()*100:.2f}"

In [ ]:
# assemblies = pd.read_csv('/net/mraid20/export/jafar/Microbiome/Analyses/Unicorn/URS/URS_Build/full_metadata_filtered_w_clusts_reps.csv', index_col=0)
# assemblies.index = assemblies['AssemblyName'].str.replace('.fa', '').str.replace('.fna', '').values

In [ ]:
# quality = assemblies.join(quality, lsuffix=' checkM1', rsuffix=' checkM2')
# # quality = quality[quality['cluster_s'].isin(all_species)]

In [ ]:
# quality = quality[['Completeness checkM1', 'Contamination checkM1', 'N50 (contigs) checkM1', 'Score_1_-5_15',
#                    'cluster_s', 'is_rep',
#                    'Completeness checkM2', 'Contamination checkM2', 'N50 (contigs) checkM2']].astype(float)

In [ ]:
# f"70% {((quality['Completeness checkM2'] < 70) | (quality['Contamination checkM2'] > 5)).mean()*100:.2f}"
# f"50% {((quality['Completeness checkM2'] < 50) | (quality['Contamination checkM2'] > 10)).mean()*100:.2f}"
 
# for q in ['Completeness', 'Contamination']:#, 'N50 (contigs)']:
#     q, spearmanr(quality[f'{q} checkM1'], quality[f'{q} checkM2'])
#     if q == 'Completeness':
#         f"70% {(quality['Completeness checkM2'] < 70).mean()*100:.2f}"
#         f"50% {(quality['Completeness checkM2'] < 50).mean()*100:.2f}"
#     if q == 'Contamination':
#         f"5% {(quality['Contamination checkM2'] > 5).mean()*100:.2f}"
#         f"10% {(quality['Contamination checkM2'] > 10).mean()*100:.2f}"
#     _ = plt.figure()
#     _ = sns.scatterplot(x=f'{q} checkM1', y=f'{q} checkM2', data=quality, alpha=0.5)

In [ ]:
# gunc = pd.read_csv('/net/mraid20/ifs/wisdom/segal_lab/jafar/Microbiome/Analyses/Unicorn/URS/URS_Build/Rep_all/gunc/GUNC_all.csv')
# gunc['pass.GUNC'].mean()*100

# gunc.columns
# # plt.figure()
# # (gunc['contamination_portion']*100).hist()
# # plt.figure()
# # (gunc['clade_separation_score']*100).hist()

# # gunc = gunc.set_index('genome')[['genes_retained_index', 'contamination_portion']]*100
# # gunc.columns = ['Completeness GUNC', 'Contamination GUNC']
# # gunc.index = gunc.index.str.split('_').str[-1].astype(float)
# # gunc = quality[quality['is_rep'] == 1].set_index('cluster_s').join(gunc)

In [ ]:
# for q in ['Completeness checkM1', 'Completeness checkM2', 'Contamination checkM1', 'Contamination checkM2']:
#     q, spearmanr(gunc[f'{q.split(" ")[0]} GUNC'], gunc[q])
#     if q == 'Completeness checkM1':
#         f"70% {(gunc['Completeness GUNC'] < 70).mean()*100:.2f}"
#         f"50% {(gunc['Completeness GUNC'] < 50).mean()*100:.2f}"
#     if q == 'Contamination checkM1':
#         f"5% {(gunc['Contamination GUNC'] > 5).mean()*100:.2f}"
#         f"10% {(gunc['Contamination GUNC'] > 10).mean()*100:.2f}"
#     _ = plt.figure()
#     _ = sns.scatterplot(x=f'{q.split(" ")[0]} GUNC', y=q, data=gunc, alpha=0.5)

## genes

In [ ]:
# # calculate genes_per_percentile of genomes
# # calculate genes_per_genomes as the average of 10 random orders of genomes
# # create the prokka annotation file with the %genomes the genes are in

# indices = np.arange(1, 11)
# genes_per_genomes = {i: pd.DataFrame(index=np.arange(metadata['#assemblies'].max()).astype(int), columns=all_species, dtype=int) for i in indices}

# prec = {}
# genes_per_percentile = pd.DataFrame(index=pd.cut(pd.DataFrame(np.arange(1, 101))[0], bins=np.arange(0, 105, 5)).drop_duplicates().values, dtype=int)

# for species in all_species:
#     genes = read_gene_presence_absence(species)
    
#     for i in indices:
#         genes = genes.sample(frac=1, random_state=i, axis=1)
#         genes_per_genomes[i].loc[np.arange(genes.shape[1]), species] = (np.cumsum(genes, axis=1) > 0).sum().values
    
#     genes = genes.sum(axis=1)/genes.shape[1]*100  # taking the last order, order does not matter when you sum overall
#     prec[species] = genes.copy()
#     genes = pd.cut(genes, bins=np.arange(0, 105, 5)).value_counts().astype(int)
#     genes_per_percentile.loc[genes.index, species] = genes.values
    
# genes_per_percentile.to_pickle(os.path.join('data_frames', f'genes_per_percentile.df'))

# for i in indices:
#     genes_per_genomes[i].to_pickle(os.path.join('data_frames', f'genes_per_genomes_random_order{i}.df'))

# new = None
# for df in genes_per_genomes.values():
#     new = new + df if new is not None else df
# genes_per_genomes = new/len(indices)
# genes_per_genomes.to_pickle(os.path.join('data_frames', f'genes_per_genomes_random_order.df'))

# all_prec = []
# for species, v in prec.items():
#     all_prec.append(v.to_frame('%genomes').assign(SGB=species))
# all_prec = pd.concat(all_prec).reset_index().set_index(['SGB', 'Gene'])
# all_prec.reset_index().to_csv(os.path.join('run', 'Segal_annots_all_assemblies_prokka_percentage.csv'))

## core, shell, cloud

In [ ]:
# # label the genes_per_percentile as core, shell and cloud genes
# genes_type = genes_per_percentile.sort_index()
# genes_type = genes_type.iloc[-2:].sum().to_frame('Core genes').join(
#              genes_type.iloc[2:-2].sum().to_frame('Shell genes')).join(
#              genes_type.iloc[:2].sum().to_frame('Cloud genes')).join(
#              genes_type.sum().to_frame('All genes'))
# genes_type = genes_type.join(metadata.set_index('species')['#assemblies'].dropna().astype(int).to_frame('n_genomes'))

# # add the best genome information
# genes_type = genes_type.join(genome_annots.groupby('Rep').apply(len).to_frame('Genes in best genome'))
# genes_type['Fold change'] = genes_type['All genes']/genes_type['Genes in best genome']

# # compare the pangenome of 20 individual genomes to reduce bias
# genes_type['Genes in 20'] = genes_per_genomes.iloc[19].loc[genes_type.index]
# genes_type['Fold change 20'] = genes_type['Genes in 20']/genes_type['Genes in best genome']

# genes_type.to_pickle(os.path.join('data_frames', f'genes_type.df'))

In [ ]:
# # create the prokka annotation file with the %genomes the genes are in OUT OF 20 GENOMES

# indices = np.arange(1, 11)

# for i in indices:

#     prec = {}

#     for species in all_species:
#         genes = read_gene_presence_absence(species)
#         genes = genes.sample(frac=1, random_state=i, axis=1)
#         genes = genes.iloc[:, :20]  # OUT OF 20 GENOMES
#         genes = genes.sum(axis=1)/genes.shape[1]*100
#         prec[species] = genes.copy()

#     all_prec = []
#     for species, v in prec.items():
#         all_prec.append(v.to_frame('%genomes').assign(SGB=species))
#     all_prec = pd.concat(all_prec).reset_index().set_index(['SGB', 'Gene'])
#     all_prec.reset_index().to_csv(os.path.join('run', f'Segal_annots_20_assemblies_random_order_{i}_prokka_percentage.csv'))

In [ ]:
# indices = np.arange(1, 11)

# jaccard = pd.DataFrame(index=all_species, columns=indices)

# full = get_sub_annots(pan_annots).set_index('Gene', append=True).assign(full=True)[['full']]

# for i in indices:
#     df = pd.read_csv(os.path.join('run', f'Segal_annots_20_assemblies_random_order_{i}_prokka_percentage.csv'), index_col=0)
#     df = get_sub_annots(df).assign(twenty=True).set_index('Gene', append=True)[['twenty']]
#     df = df.join(full, how='outer').fillna(False)
#     df = df.groupby('SGB').apply(lambda s: s.all(axis=1).mean())
#     jaccard.loc[all_species, i] = df.loc[all_species]
    
# jaccard.to_pickle(os.path.join('data_frames', f'jaccard20.df'))

In [ ]:
# jaccard_full = pd.DataFrame(index=all_species, columns=all_species)

# full = get_sub_annots(pan_annots).set_index('Gene', append=True).assign(full=True)[['full']]

# for i, s1 in enumerate(all_species):
#     for s2 in all_species[i+1:]:
#         jaccard_full.loc[s1, s2] = full.loc[s1].join(full.loc[s2], how='outer', rsuffix='2').fillna(False).all(axis=1).mean()
        
# jaccard_full.stack().to_pickle(os.path.join('data_frames', f'jaccard_full.df'))

In [ ]:
# indices = np.arange(1, 11)

# shell_r = pd.DataFrame(index=all_species, columns=indices)
# shell_p = pd.DataFrame(index=all_species, columns=indices)

# full = get_sub_annots(pan_annots, 10, 90).set_index('Gene', append=True).assign(full=True)[['full']]

# for i in indices:
#     df = pd.read_csv(os.path.join('run', f'Segal_annots_20_assemblies_random_order_{i}_prokka_percentage.csv'), index_col=0)
#     df = get_sub_annots(df, 10, 90).assign(twenty=True).set_index('Gene', append=True)[['twenty']]
#     df = df.join(full, how='outer').fillna(0)
#     df = df.groupby('SGB').apply(lambda s: spearmanr(s['full'], s['twenty']))
#     shell_r.loc[all_species, i] = df.loc[all_species].str[0]
#     shell_p.loc[all_species, i] = df.loc[all_species].str[1]
    
# shell_r.to_pickle(os.path.join('data_frames', f'shell_r20.df'))
# shell_p.to_pickle(os.path.join('data_frames', f'shell_p20.df'))

In [ ]:
# genes_per_per = pd.read_pickle(os.path.join('data_frames', f'genes_per_percentile.df'))

In [ ]:
# data = genes_per_per.iloc[-3:].stack().reset_index()
# data.columns = ['% Strains', 'Species', 'Number of genes']
# data['% Strains'] = data['% Strains'].astype(str)
# data['Number of genes'] = data['Number of genes'].clip(upper=1500)
# _ = sns.histplot(x='Number of genes', hue='% Strains', hue_order=data['% Strains'].unique(), data=data, palette='Reds', element='step', binwidth=100)

# plt.savefig(os.path.join('figs', '4reviewer - core threshold'))

## strain variability

In [ ]:
# data = genes_per_genomes.stack().reset_index()#.iloc[:19]
# data.columns = ['n_genomes', 'species', 'n_genes']
# data['n_genomes'] = data['n_genomes'] + 1

In [ ]:
# def heap_function(n, k, a):
#     return k * n ** a

# heaps = pd.DataFrame(index=data['species'].unique(), columns=['k', 'a', 'k_std_err', 'a_std_err'])

# for species in data['species'].unique():
#     genomes = data.loc[data['species'] == species, 'n_genomes']
#     genes = data.loc[data['species'] == species, 'n_genes']

#     params, covariance = curve_fit(heap_function, genomes, genes)

#     heaps.loc[species, ['k', 'a']] = params
#     heaps.loc[species, ['k_std_err', 'a_std_err']] = np.sqrt(np.diag(covariance))
    
# heaps = heaps.astype(float)

# heaps.to_pickle(os.path.join('data_frames', f'heaps.df'))#20

In [ ]:
# heaps_genes = pd.DataFrame(columns=['s', 'p'])
# for gene in core_annots['Gene'].unique():
#     have_gene = core_annots[core_annots['Gene'] == gene].index.to_list()
#     miss_gene = list(set(all_species) - set(have_gene))
#     if (len(have_gene) >= 30) & (len(miss_gene) >= 30):
#         heaps_genes.loc[gene] = ttest_ind(heaps.loc[have_gene, 'a'], heaps.loc[miss_gene, 'a'])
# heaps_genes['p_bonferroni'] = (heaps_genes['p']*heaps_genes.shape[0]).clip(upper=1)
# heaps_genes.to_csv(os.path.join('data_frames', 'heaps_genes.csv'))

## antibiotic resistances

In [ ]:
# # calculate the %strains with antibiotic resistence per species
# anti = pan_annots.reset_index()[['SGB', 'Gene', 'GENE', 'PRODUCT', 'RESISTANCE']].dropna()

# # multiply rows with several resistances
# multiple = []
# for i, row in anti[anti['RESISTANCE'].str.contains(';')].iterrows():
#     for res in row['RESISTANCE'].split(';'):
#         new = row.copy()
#         new['RESISTANCE'] = res
#         multiple.append(new)
# multiple = pd.concat(multiple, axis=1).T

# names = ['polymyxin|colistin', 'aztreonam', 'tigecycline', 'fosfomycin', 'daptomycin', 'linezolid', 'vancomycin']
# # 'carbapenem|meropenem' and 'cephalosporin' are excluded because they are antibiotic classes
# # some cephalosporin, especially 4th and 5th generation should be taken, however the annotations are not specific enough to distinguish
# last_resort = []
# for name in names:
#     for i, row in anti[anti['GENE'].str.lower().str.contains(name) | anti['PRODUCT'].str.lower().str.contains(name)].iterrows():
#         new = row.copy()
#         new['RESISTANCE'] = name
#         last_resort.append(new)
# last_resort = pd.concat(last_resort, axis=1).T

# anti = pd.concat([anti[~anti['RESISTANCE'].str.contains(';')], multiple, last_resort])
# anti = anti[['SGB', 'Gene', 'RESISTANCE']]

# def get_prec(s):
#     df = read_gene_presence_absence(s.name)
#     return s.groupby('RESISTANCE').apply(lambda r: df.loc[r['Gene']].any().mean()*100)

# anti = anti.groupby('SGB').apply(get_prec).unstack('RESISTANCE').fillna(0)
# anti = pd.concat([anti, pd.DataFrame(data=0, index=list(set(all_species) - set(anti.index)), columns=anti.columns)])
# anti = anti.sort_index().T.sort_index()

# anti.to_pickle(os.path.join('data_frames', 'antibiotics.df'))

# genome set

In [ ]:
# base_path = 'data_frames_i90'
# comp_path = 'data_frames_i95'
# # comp_path = 'data_frames_not_short'

In [ ]:
# base = pd.read_pickle(os.path.join(base_path, 'genes_type.df')).join(pd.read_pickle(os.path.join(base_path, 'heaps.df')))
# comp = pd.read_pickle(os.path.join(comp_path, 'genes_type.df')).join(pd.read_pickle(os.path.join(comp_path, 'heaps.df')))

In [ ]:
# base.corrwith(comp, method='spearman').to_frame(f'corr_{comp_path.replace("data_frames_", "")}').join(
#     base.mean().to_frame('base_mean')).join(
#     base.std().to_frame('base_std')).join(
#     comp.mean().to_frame(f'{comp_path.replace("data_frames_", "")}_mean')).join(
#     comp.std().to_frame(f'{comp_path.replace("data_frames_", "")}_std')).join(
#     (comp.mean()-base.mean()).to_frame(f'{comp_path.replace("data_frames_", "")}_diff'))    

In [ ]:
# heaps_base = pd.read_csv(os.path.join('data_frames_i90', 'heaps_genes.csv'), index_col=0)
# heaps_comp = pd.read_csv(os.path.join('data_frames_i95', 'heaps_genes.csv'), index_col=0)

In [ ]:
# heaps_base.shape
# heaps_comp.shape

In [ ]:
# heaps_base.index.intersection(heaps_comp.index)
# heaps_base.index.difference(heaps_comp.index)
# heaps_comp.index.difference(heaps_base.index)

In [ ]:
# heaps_base[heaps_base['p_bonferroni'] < 0.05].shape
# heaps_comp[heaps_comp['p_bonferroni'] < 0.05].shape

In [ ]:
# heaps_base[heaps_base['p_bonferroni'] < 0.05].index.intersection(heaps_comp[heaps_comp['p_bonferroni'] < 0.05].index)
# heaps_base[heaps_base['p_bonferroni'] < 0.05].index.difference(heaps_comp[heaps_comp['p_bonferroni'] < 0.05].index).shape#.tolist()
# heaps_comp[heaps_comp['p_bonferroni'] < 0.05].index.difference(heaps_base[heaps_base['p_bonferroni'] < 0.05].index)